In [17]:
# Install dependencies
!pip install groq==0.13.1 PyPDF2==3.0.1 pandas==2.2.2

In [18]:
from google.colab import userdata
GROQ_API_KEY=userdata.get('GROQ_API_KEY')

In [19]:
# Import necessary libraries
import os
import re
import pandas as pd
from PyPDF2 import PdfReader
from groq import Groq


In [20]:
# Initialize the Groq client
client = Groq(api_key=GROQ_API_KEY)

In [25]:
#create resumes folder with atleast 10 pdf resume files
#you will get ats_resluts.csv in the end.
#Install dependencies
#!pip install groq python-dotenv PyPDF2 pandas

# # Import necessary libraries
# import os
# import re
# import pandas as pd
# from PyPDF2 import PdfReader
# from groq import Groq
# from google.colab import userdata

# # Set your Groq API key using Colab's userdata
# GROQ_API_KEY = userdata.get('GROQ_API_KEY')

# # Initialize the Groq client with the Colab userdata key
# client = Groq(api_key=GROQ_API_KEY)

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
        return text
    except Exception as e:
        return f"Error reading {pdf_path}: {str(e)}"

# Function to extract email, name, and phone number from resume text
def extract_email_name_phone(text):
    # Extract email using regex
    email_match = re.search(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", text)
    email = email_match.group() if email_match else "Not Found"

    # Refined phone number regex (covers more formats and avoids capturing year-like patterns)
    phone_match = re.search(r"(\+?\d{1,4}[\s-]?)?(\(?\d{1,3}\)?[\s-]?)?(\d{1,4}[\s-]?)?(\d{1,4}[\s-]?)?(\d{1,4})", text)

    # Check for valid phone numbers: Filter out years or non-phone-like numbers
    phone_number = phone_match.group() if phone_match else "Not Found"
    if re.match(r"\d{4}", phone_number):  # If it's a 4-digit number (like "2022"), consider it invalid
        phone_number = "Not Found"

    # Improved name extraction: Searching for common name patterns or using the first line as fallback
    name_match = re.search(r"(?:Name|Full\s*Name)[\s:]*([A-Za-z\s]+)", text, re.IGNORECASE)

    if name_match:
        name = name_match.group(1).strip()
    else:
        # Attempt to extract the name by considering the first non-empty line
        name = next((line.strip() for line in text.splitlines() if line.strip()), "Name not found")

    return name, email, phone_number

# Function to get match score using Groq API
def get_match_score(jd, resume_text, model="llama3-8b-8192"):
    prompt = f"""
    Job Description: {jd}
    Resume: {resume_text}

    Please provide:
    1. A match score (0-100) between the job description and resume.
    2. Missing keywords from the resume based on the job description.
    3. A one-sentence summary of the applicant.
    """
    try:
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=model,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Main ATS processing function
def process_bulk_resumes(jd, pdf_folder):
    results = []
    for file in os.listdir(pdf_folder):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, file)
            print(f"Processing {file}...")

            # Extract text, email, name, and phone number
            resume_text = extract_text_from_pdf(pdf_path)
            name, email, phone_number = extract_email_name_phone(resume_text)

            # Get match score and LLM feedback
            llm_response = get_match_score(jd, resume_text)
            match_score_match = re.search(r"match score.*?(\d{1,3})", llm_response, re.IGNORECASE)
            match_score = int(match_score_match.group(1)) if match_score_match else "N/A"

            # Append to results
            results.append({"Name": name, "Email": email, "Phone Number": phone_number, "Match Score": match_score})

    # Save results to CSV
    df = pd.DataFrame(results)
    df.to_csv("ats_results.csv", index=False)
    print("Results saved to ats_results.csv")

# Example Usage
# Prompt user to input JD (Job Description)
jd = input("Please enter the Job Description: ")

# Upload resumes to Colab folder, set JD, and run
pdf_folder = "/content/resumes"  # Replace with the path to your resumes folder

# Process resumes
process_bulk_resumes(jd, pdf_folder)


Please enter the Job Description: Full job description We are a software development company with a strong specialised focus on developing artificial intelligence-based solutions for real-world industrial problems. We focus on using state-of-the-art AI technologies to solve complex industrial problems.  We are looking for a Machine Learning (ML) engineer, with a specialisation in deep learning, to join our team.  Position Details  Type: Full-time  Location: In-person, Johar Town Lahore  Job Description  Coordinating with an in-house or external data team to design a strategy for collecting appropriate data for building a deep learning model for the required task  Designing, building and training deep learning models using state-of-the-art architectures including CNNs, LSTMs, attention networks and transformers. Training is either done on our on-premise servers or on the cloud (e.g., using AWS servers)  Deploying the final model on a server and providing an interface or APIs to interact